# Hands-on: Getting Started with Generative AI APIs
### by Mike Wolfson
### **DevFest Waterloo** - 10/21/2023

In this hands-on workshop, participants will embark on a journey into the realm of generative AI, harnessing the power of Jupyter Notebook. We'll demystify the foundations of generative AI, and get our hands dirty with real-time experiments. Whether you're a budding data scientist, or just intrigued by this emerging technology, join us as we unravel the potential of generative AI, one API call at a time. Prepare to leave with the knowledge, experience, and tools to craft your own generative masterpieces!



## Python

```
Python's simplicity, extensive libraries, community support, and versatility make it an ideal choice for AI development. It provides the tools and resources necessary to build AI models, work with data, and deploy AI solutions in a wide range of applications.
```

Download and Install Python [Site](https://www.python.org/downloads/)

Official Documentation [Site](https://docs.python.org/3/)

## PIP

**Pythons Package Manager** is now installed 

*We will be using this to download and install all of our project dependencies*

## Jupyter Notebook

- Introduction to Jupyter Notebooks.
- Benefits of using Jupyter for interactive development.
- Understanding cells, outputs, and extensions.

[Jupyter Documentation](https://docs.jupyter.org/en/latest/)


### PIP Install

 `pip install jupyter`

## Hugging Face Transformers API

PIP Install: 

`pip install transformers torch`

Documentation: [Install Instructions](https://huggingface.co/docs/transformers)

### generate_text_huggingface: helper function

Use this to execute calls to the Hugging Face Transformers API

*No API Key needed!*

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_text_huggingface(prompt, model_name="gpt2-medium"):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, temperature=1.0)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

## Sending a prompt to the **Hugging Face Helper Function**

In [ ]:
print(generate_text_huggingface("Once upon a time"))

## Working with the Google PaLM APIs

```
Large Language Models (LLMs) are a powerful, versatile type of machine learning model that enables computers to comprehend and generate natural language better than ever. They can be used to build all sorts of applications, from chat bots to virtual assistants to translation apps and much more. Plus, you don't have to be an AI expert or even write code to use them. All it takes are a few sentences or “prompts” to get started designing your own custom LLM app.
```

**PIP** - Install dependencies:

`pip install -q google.generativeai`

API key and documentation: [developers.generativeai.google](http://developers.generativeai.google)

In [ ]:
import pprint
import google.generativeai as palm

### Working with API keys

You will need to set the PaLM API key as a system variable named: `GOOGLE_API_KEY`.  

- [Setting an Environment Variable on Mac/Linux](https://phoenixnap.com/kb/set-environment-variable-mac)
- [Setting an Environment Variable on Windows](https://phoenixnap.com/kb/windows-set-environment-variable)

PIP:

`pip install python-dotenv`

In [ ]:
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
apiKey = os.getenv('GOOGLE_API_KEY')

palm.configure(api_key=apiKey,
               transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    ))

## Explore the Available Models

In [ ]:
for m in palm.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

### Filter models by their supported generation methods
- `generateText` is currently recommended for coding-related prompts.
- `generateMessage` is optimized for multi-turn chats (dialogues) with an LLM.

In [ ]:
models = [m for m in palm.list_models()
          if 'generateText'
          in m.supported_generation_methods]
models

In [ ]:
model_bison = models[0]
model_bison

### generate_text_palm: helper function to generate text

- The `@retry` decorator helps you to retry the API call if it fails.
- We set the temperature to 0.0 so that the model returns the same output (completion) if given the same input (the prompt).).

In [ ]:
from google.api_core import retry
@retry.Retry()
def generate_text_palm(prompt,
                  model=model_bison,
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

## Sending a prompt to the **PaLM Helper Function**

In [ ]:
completion = generate_text_palm("Saturdays are perfect for")
print(completion.result)

## Working with the Open AI APIs

```
OpenAI's APIs offer developers the ability to integrate advanced artificial intelligence capabilities into their applications, enabling a wide range of tasks from text generation to complex problem-solving.
```
Documentation: [https://beta.openai.com/docs/](https://beta.openai.com/docs/)

### Obtaining API Keys:
- **OpenAI Platform**: [https://platform.openai.com/](https://platform.openai.com/)
  - After signing up or logging in, navigate to the API section to manage and obtain your API keys.
- You will need to set the PaLM API key as a system variable named: `OPENAI_API_KEY`.  

Note: do NOT check your API key into a public Github repo, or it will get revoked 
  
  


### PIP Dependencies

`pip install openai`

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### generate_text_openai: helper function to generate text from Open AI API

In [ ]:
def generate_text_openai(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Sending a prompt to the **Open AI Helper Function**

In [ ]:
print(generate_text_openai("You are Igor the pirate"))


## Prompt template

Using this simple template will help you organize your Prompts more effectively.

- pre: establishing context about what the quesiton is about ("you are a pirate")
- question: what we are asking the GenAI to do ("you are telling us your favorite joke about booty")
- post: formatting and other after-thoughts ("use kid friendly language, and output the results in a json format")

I learned this prompt technique in the course "[Pair Programming with a Large Language Model](https://www.deeplearning.ai/short-courses/pair-programming-llm/)" - this is a short course by Laurence Moroney, and is a free course from [Deeplearning.ai](https://www.deeplearning.ai/)

In [ ]:
prompt_template = """s
{pre}

{question}

{post}
"""

In [ ]:
pre = """
You are Igor the pirate.
"""


In [ ]:
question = """
tell us your favorite joke about booty
"""

In [ ]:
post = """
use child friendly language, and output the results in a json format
"""

In [ ]:
prompt = prompt_template.format(pre=pre, question=question, post=post)
print(generate_text_openai(prompt))